In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from time import sleep, time
from random import uniform

In [2]:
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(options=chrome_options)

keywords = ['subreddit', 'wikipedia', 'subreddit moderation']

# the returned number is unstable, need multiple clicks
clicks = 5

# the wait time needs to be randomized so the scraper is not identified as a bot
# the range is uniformly sampled between [a, b] seconds
# the time also needs to be large enough so the search results could be returned
a = 0.8
b = 1.2

# with the current parameters, about 20 seconds per keyword
for keyword in keywords:
    
    start = time()

    url = f"https://aclanthology.org/search/?q={keyword.replace(' ', '+')}"

    # start the driver
    driver.get(url)
    
    nums = []

    # switch between the options multiple times
    for _ in range(clicks):
        
        # expand the dropdown menu
        buttons = driver.find_elements(by=By.CLASS_NAME, value="gsc-selected-option-container")
        buttons[0].click()
    
        # choose year of publication
        buttons =  driver.find_elements(by=By.CLASS_NAME, value="gsc-option-menu-item")
        buttons[1].click()

        # some time is needed for the result to change
        sleep(uniform(a,b))

        soup = BeautifulSoup(driver.page_source, "html.parser")

        # result in the format "About 31 results (0.11 seconds)"
        result = soup.find_all("div", {"class": "gsc-result-info"})

        # get the number
        num = result[0].string.split(' ')[1].replace(',', '')
        nums.append(num)
        
        # expand the dropdown menu
        buttons = driver.find_elements(by=By.CLASS_NAME, value="gsc-selected-option-container")
        buttons[0].click()
        
        # change back to relevance
        buttons =  driver.find_elements(by=By.CLASS_NAME, value="gsc-option-menu-item")
        buttons[0].click()
        sleep(uniform(a,b))
        
    end = time()
        
    print(keyword, min(nums), f"({end-start:.2f} seconds)")

# quit the driver
driver.quit()

subreddit 31 (15.06 seconds)
wikipedia 1900 (11.40 seconds)
subreddit moderation 2 (11.17 seconds)
